# Setting up the Fast Withdrawal contract:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `230.790527 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.240557203 ꜩ`


## Deploy FastWithdrawal:

In [2]:
from scripts.tezos import deploy_fast_withdrawal

fast_withdrawal = deploy_fast_withdrawal.callback(
    xtz_ticketer_address=XTZ_TICKETER_ADDRESS,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Native XTZ Ticketer address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
Successfully deployed Fast Withdrawal, address: `KT1KYcb2G4M3UCeDNcDr8TghDfHLeqmdKMqg`


## Making Fast Withdrawal on Etherlink side:

In [3]:
from scripts.etherlink import xtz_fast_withdraw

tx_hash = xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=717 * 10**12,
    discounted_amount=700,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)
tx_hash

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1KYcb2G4M3UCeDNcDr8TghDfHLeqmdKMqg`
      * Payload bytes: `0500bc0a`
      * Amount (mutez): `717`
      * Discounted amt (mutez): `700`
      * Fee (mutez): `17`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0x21fc4b49906ad5ea62814f144b7d0ba47dd46ba02bea0a2a2a54a4f0a8d229af`


'0x21fc4b49906ad5ea62814f144b7d0ba47dd46ba02bea0a2a2a54a4f0a8d229af'

### Searching for outbox message:
- TODO: Move this code to a separate script

In [64]:
from scripts.helpers.rollup_node import get_messages
import time
from pytezos import MichelsonType
from pytezos import michelson_to_micheline


def make_event_logs_link(tx_hash):
    return f'https://testnet.explorer.etherlink.com/tx/{tx_hash}?tab=logs'

def make_outbox_message_link(level):
    return f'https://ghostnet-smart.tzkt.io/global/block/{level}/outbox/{level}/messages'

def make_tzkt_ghostnet_link(tx_hash):
    return f'https://ghostnet.tzkt.io/{tx_hash}'

FAST_WITHDRAWAL_OUTBOX_INTERFACE = '''
pair %default (nat %withdrawal_id)
    (pair (ticket %ticket (pair nat (option bytes)))
         (pair (timestamp %timestamp)
               (pair (address %base_withdrawer)
                     (pair (bytes %payload) (bytes %l2_caller)))))
'''

def decode_outbox_message(message):
    micheline_expression = michelson_to_micheline(FAST_WITHDRAWAL_OUTBOX_INTERFACE)
    michelson_type = MichelsonType.match(micheline_expression)
    assert len(message['transactions']) == 1
    parameters_micheline = message['transactions'][0]['parameters']
    return michelson_type.from_micheline_value(parameters_micheline).to_python_object()

def scan_outbox_until_message_found(
    etherlink_rollup_node_url: str,
    last_level: int,
    max_levels: int = 100,
    sleep_time: int = 0.3
):
    click.echo('Scanning outbox: ', nl=False)
    for level in range(last_level, last_level-max_levels, -1):
        click.echo(wrap(accent(level)), nl=False)
        messages = get_messages(ETHERLINK_ROLLUP_NODE_URL, level)
        if len(messages) > 0:
            click.echo()
            click.echo('Found outbox message at level: ' + wrap(accent(level)))
            return messages
        click.echo(', ', nl=False)
        time.sleep(sleep_time)

def make_withdrawal_info(tx_hash, outbox_message):
    click.echo('Fast Withdrawal info:')
    click.echo('- Event logs: ' + make_event_logs_link(tx_hash))
    click.echo('- Outbox message: ' + make_outbox_message_link(tx_hash))
    click.echo('- Withdrawal ID: ' + wrap(accent(outbox_message['withdrawal_id'])))
    click.echo('- Timestamp: ' + wrap(accent(outbox_message['timestamp'])))

last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Scanning outbox: `11607370`, `11607369`, `11607368`, `11607367`, `11607366`
Found outbox message at level: `11607366`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0x21fc4b49906ad5ea62814f144b7d0ba47dd46ba02bea0a2a2a54a4f0a8d229af?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0x21fc4b49906ad5ea62814f144b7d0ba47dd46ba02bea0a2a2a54a4f0a8d229af/outbox/0x21fc4b49906ad5ea62814f144b7d0ba47dd46ba02bea0a2a2a54a4f0a8d229af/messages
- Withdrawal ID: `1941`
- Timestamp: `1743758058`


## Purchasing withdrawal on Tezos side:

In [40]:
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.ticket_content import TicketContent

discounted_amount = 700

withdrawal = Withdrawal(
    withdrawal_id=outbox_message['withdrawal_id'],
    full_amount=717,
    ticketer=XTZ_TICKETER_ADDRESS,
    content=TicketContent(
        token_id=0,
        token_info=None,
    ),
    timestamp=outbox_message['timestamp'],
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=discounted_amount,
).send()

click.echo('- Transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

- Transaction: https://ghostnet.tzkt.io/onpg49p34ncDv6AadaNghpyS8rU9oyErNmXgKWzCHKRM8FqKrMz


## Trying to purchase twice:
- Forcing the transaction to be executed

In [41]:
opg = (
    fast_withdrawal.payout_withdrawal(
        withdrawal=withdrawal,
        service_provider=get_address(tezos_account),
        xtz_amount=discounted_amount,
    )
    .as_transaction()
    .fill()
    .sign()
    .inject()
)

click.echo('- Transaction: ' + make_tzkt_ghostnet_link(opg['hash']))

- Transaction: https://ghostnet.tzkt.io/ooDg9ysEUchGxugx1BLV7iHiNy7t2ocymkyaYJ74dYFpXvNPcnA


## Making Additional Withdrawals:
### One with an incorrect payload that would be impossible to execute

In [42]:
from scripts.etherlink.xtz_fast_withdraw import (
    load_withdraw_precompile,
    make_fast_withdrawal,
)

precompile_contract = load_withdraw_precompile(
    XTZ_WITHDRAWAL_PRECOMPILE,
    web3
)
receipt = make_fast_withdrawal(
    etherlink_account=etherlink_account,
    web3=web3,
    precompile_contract=precompile_contract,
    target=get_address(tezos_account),
    fast_withdrawals_contract=fast_withdrawal.address,
    payload=bytes.fromhex('ff4f'),
    wei_amount=133 * 10**12,
)

click.echo(
    'Successfully initiated wrong XTZ Fast Withdrawal, tx hash: '
    + wrap(accent(receipt.transactionHash.hex()))
)

Successfully initiated wrong XTZ Fast Withdrawal, tx hash: `0x93439dbcbaca84ef5bde8b59b93c87098f1b3df9393034c12ef20560cdf7f1a1`


### Three more with different base withdrawers and amounts

In [43]:
from scripts.etherlink import xtz_fast_withdraw
from random import randint

tx_hash = xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1KYcb2G4M3UCeDNcDr8TghDfHLeqmdKMqg`
      * Payload bytes: `05009009`
      * Amount (mutez): `341`
      * Discounted amt (mutez): `592`
      * Fee (mutez): `-251`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0x8fd31710cbca4a6816bfb6df16edbdb6dd6be49a26d89c8944d2bc24c1a4d311`


'0x8fd31710cbca4a6816bfb6df16edbdb6dd6be49a26d89c8944d2bc24c1a4d311'

In [70]:
time.sleep(20)
last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Scanning outbox: `11607769`, `11607768`
Found outbox message at level: `11607768`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0x8fd31710cbca4a6816bfb6df16edbdb6dd6be49a26d89c8944d2bc24c1a4d311?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0x8fd31710cbca4a6816bfb6df16edbdb6dd6be49a26d89c8944d2bc24c1a4d311/outbox/0x8fd31710cbca4a6816bfb6df16edbdb6dd6be49a26d89c8944d2bc24c1a4d311/messages
- Withdrawal ID: `1943`
- Timestamp: `1743759716`


In [71]:
tx_hash = xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1bBaMePLESdyn1ueewzZNZ9o7d1zk8XjBZ`
      * Fast Withdrawal contract: `KT1KYcb2G4M3UCeDNcDr8TghDfHLeqmdKMqg`
      * Payload bytes: `0500830c`
      * Amount (mutez): `970`
      * Discounted amt (mutez): `771`
      * Fee (mutez): `199`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0x4a4fbe7aba71db08f7e44e363e407db4c323661b7d86cc2de98c4546d0cd6788`


In [73]:
time.sleep(20)
last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Scanning outbox: `11610385`, `11610384`
Found outbox message at level: `11610384`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0x4a4fbe7aba71db08f7e44e363e407db4c323661b7d86cc2de98c4546d0cd6788?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0x4a4fbe7aba71db08f7e44e363e407db4c323661b7d86cc2de98c4546d0cd6788/outbox/0x4a4fbe7aba71db08f7e44e363e407db4c323661b7d86cc2de98c4546d0cd6788/messages
- Withdrawal ID: `1944`
- Timestamp: `1743770552`


In [74]:
tx_hash = xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

time.sleep(20)
last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1b5QtUwpfBBoNyHn6Neyq3vxuNGkHpqoPS`
      * Fast Withdrawal contract: `KT1KYcb2G4M3UCeDNcDr8TghDfHLeqmdKMqg`
      * Payload bytes: `05009e02`
      * Amount (mutez): `169`
      * Discounted amt (mutez): `158`
      * Fee (mutez): `11`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xab015d054fafcb7b691521c22e57fa05db61a484949f5bfc7cd15f03f846c258`
Scanning outbox: `11610409`, `11610408`, `11610407`, `11610406`, `11610405`, `11610404`, `11610403`, `11610402`, `11610401`, `11610400`, `11610399`, `11610398`, `11610397`, `11610396`
Found outbox message at level: `11610396`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0xab015d054fafcb7b691521c22e57fa05db61a484949f5bfc7cd15f03f846c258?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0xab015d0